# Client

Demo of client interacting with the simple chain server, which deploys a chain that tells jokes about a particular topic.

You can interact with this via API directly

In [1]:
import requests

inputs = {"input": {"topic": "sports"}}
response = requests.post("http://localhost:8000/invoke", json=inputs)

response.json()

{'output': "Why don't scientists trust atoms who play sports?\n\nBecause they make up everything!",
 'callback_events': []}

You can also interact with this via the RemoteRunnable interface (to use in other chains)

In [ ]:
from langserve import RemoteRunnable

remote_runnable = RemoteRunnable("http://localhost:8000/")

Remote runnable has the same interface as local runnables

In [ ]:
response = await remote_runnable.ainvoke({"topic": "sports"})

The client can also execute langchain code synchronously, and pass in configs

In [ ]:
from langchain.schema.runnable.config import RunnableConfig

remote_runnable.batch([{"topic": "sports"}, {"topic": "cars"}])

The server supports streaming (using HTTP server-side events), which can help interact with long responses in real time

In [ ]:
async for chunk in remote_runnable.astream({"topic": "bears, but a bit verbose"}):
    print(chunk, end="", flush=True)

## Configurability

The server chains have been exposed as configurable chains!

```python 

model = ChatOpenAI(temperature=0.5).configurable_alternatives(
    ConfigurableField(
        id="llm",
        name="LLM",
        description=(
            "Decide whether to use a high or a low temperature parameter for the LLM."
        ),
    ),
    high_temp=ChatOpenAI(temperature=0.9),
    low_temp=ChatOpenAI(temperature=0.1),
    default_key="medium_temp",
)
prompt = PromptTemplate.from_template(
    "tell me a joke about {topic}."
).configurable_fields(  # Example of a configurable field
    template=ConfigurableField(
        id="prompt",
        name="Prompt",
        description=("The prompt to use. Must contain {topic}."),
    )
)
```

We can now use the configurability of the runnable in the API!

In [ ]:
await remote_runnable.ainvoke(
    {"topic": "sports"},
    config={
        "configurable": {"prompt": "how to say {topic} in french", "llm": "low_temp"}
    },
)